In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")
from syft import autocache

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/shubham/PySyft


✅ The installed version of syft==0.8.1b1 matches the requirement >=0.8 and the requirement <0.8.1


In [2]:
import sys
mb = (1024 * 1024)

In [3]:

import pandas as pd
import numpy as np
from itertools import cycle
def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
    return df    

In [4]:
df = generate_fake_dataframe(size=1000000, cols =  "cififficcd")

In [5]:
# MB of data occupied in memory
sys.getsizeof(df)/mb

237.24835681915283

In [7]:
domain_client = sy.login(url="http://localhost:8081", email="info@openmined.org", password="changethis")

Logged into Lucid Brockman as <info@openmined.org>


In [8]:
domain_client.api.services

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/IPython/core/formatters.py:366: FormatterWarning: text/html formatter returned invalid type <class 'type'> (expected <class 'str'>) for object: <syft.client.api.APIModule object at 0x7f28472fbbb0>
  warnings.warn(


In [9]:
data_subjects = domain_client.api.services.data_subject.get_all()

In [10]:
data_subjects

[]

In [11]:
assert len(data_subjects) == 10

AssertionError: 

In [12]:
country = sy.DataSubject(name="Country", aliases=["country_code"])

In [13]:
canada = sy.DataSubject(name="Canada", aliases=["country_code:ca"])
germany = sy.DataSubject(name="Germany", aliases=["country_code:de"])
spain = sy.DataSubject(name="Spain", aliases=["country_code:es"])
france = sy.DataSubject(name="France", aliases=["country_code:fr"])
japan = sy.DataSubject(name="Japan", aliases=["country_code:jp"])
uk = sy.DataSubject(name="United Kingdom", aliases=["country_code:uk"])
usa = sy.DataSubject(name="United States of America", aliases=["country_code:us"])
australia = sy.DataSubject(name="Australia", aliases=["country_code:au"])
india = sy.DataSubject(name="India", aliases=["country_code:in"])

In [14]:
country.add_member(canada)
country.add_member(germany)
country.add_member(spain)
country.add_member(france)
country.add_member(japan)
country.add_member(uk)
country.add_member(usa)
country.add_member(australia)
country.add_member(india)

country.members

,key,type,id
0,Canada,<DataSubject: Canada>,None
1,Germany,<DataSubject: Germany>,None
2,Spain,<DataSubject: Spain>,None
3,France,<DataSubject: France>,None
4,Japan,<DataSubject: Japan>,None


In [15]:
registry = domain_client.data_subject_registry

In [16]:
response = registry.add_data_subject(country)

In [17]:
response

<class 'syft.service.response.SyftSuccess'>: 10 Data Subjects Registered

In [18]:
assert response

In [19]:
domain_client.data_subject_registry

,type,id
0,<DataSubject: Country>,73c904a8769d4387beb56769d4c09e5a
1,<DataSubject: France>,54543e27ded44f7cb6512798b0daab96
2,<DataSubject: Spain>,2084eef77d39450083bba016ac760fe1
3,<DataSubject: Japan>,7cae046ff3f14ba6b935bf46ff0e5f42
4,<DataSubject: United States of America>,21925673343145d1a0aff7b1ba57632e


In [20]:
data_subjects = domain_client.api.services.data_subject.get_all()

In [21]:
assert len(data_subjects) == 10

In [22]:
mock_ca_data = df

In [23]:
ca_data = df

In [24]:
from faker import Faker

faker = Faker()

## Upload Datasets

In [34]:
n_dataset = 40
n_assets = 5

for j in range(n_dataset):
    d_name = faker.name()
    print(f"Dataset {j+1}: ", d_name)
    
    dataset = sy.Dataset(name=f"{d_name} - {j+1}")
    dataset.set_description("""Canada Trade Data""")

    dataset.add_citation("Person, place or thing")
    dataset.add_url("https://github.com/OpenMined/datasets/tree/main/trade_flow")


    dataset.add_contributor(role=sy.roles.UPLOADER, 
                                    name="Andrew Trask", 
                                    email="andrew@openmined.org",
                                    note="Andrew runs this domain and prepared the dataset metadata.")

    dataset.add_contributor(role=sy.roles.EDITOR, 
                                    name="Madhava Jay", 
                                    email="madhava@openmined.org",
                                    note="Madhava tweaked the description to add the URL because Andrew forgot.")

    dataset.contributors

    assert len(dataset.contributors) == 2


    # Adding multiple assets

    ctfs = []

    for i in range(n_assets):
        ctf = sy.Asset(name=f"canada_trade_flow-{i+1}")
        ctf.set_description(f"""all the datas-{i+1}""")
        ctf.add_contributor(role=sy.roles.UPLOADER, 
                              name="Andrew Trask", 
                              email="andrew@openmined.org",
                              note="Andrew runs this domain and prepared the asset.")

        ctf.set_obj(ca_data)
        ctf.set_shape((1000000, 10))
        ctf.add_data_subject(canada)
        ctf.set_mock(mock_ca_data, mock_is_real=False)
        dataset.add_asset(ctf)
        
    domain_client.upload_dataset(dataset)
    

Dataset:  Hector Velazquez


  0%|                                                                                                                                            | 0/5 [00:00<?, ?it/s]

Uploading: canada_trade_flow-1


 20%|██████████████████████████▍                                                                                                         | 1/5 [00:03<00:14,  3.67s/it]

Uploading: canada_trade_flow-2


...

 60%|███████████████████████████████████████████████████████████████████████████████▏                                                    | 3/5 [00:09<00:06,  3.10s/it]

Uploading: canada_trade_flow-4


 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌                          | 4/5 [00:12<00:03,  3.08s/it]

Uploading: canada_trade_flow-5


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:15<00:00,  3.10s/it]


In [35]:
datasets = domain_client.api.services.dataset.get_all()

In [36]:
datasets

,type,id,name,url
0,syft.service.dataset.dataset.Dataset,3d2a1b66ad0c438cb65368976bd73599,Eric Campbell - 1,https://github.com/OpenMined/datasets/tree/mai...
1,syft.service.dataset.dataset.Dataset,679110e0b10b4a78a70f6809f4c30fc1,Jacqueline Mitchell - 2,https://github.com/OpenMined/datasets/tree/mai...
2,syft.service.dataset.dataset.Dataset,ab1ac23512924153b41cf278fe2ae9c6,Connie Davis - 3,https://github.com/OpenMined/datasets/tree/mai...
3,syft.service.dataset.dataset.Dataset,244b71ed1814490ebfc35e2f3d76d621,Benjamin Stephens - 4,https://github.com/OpenMined/datasets/tree/mai...
4,syft.service.dataset.dataset.Dataset,7dc235d551d24d2689a454b449316293,Trevor Jones - 5,https://github.com/OpenMined/datasets/tree/mai...


In [37]:
total_assets = 0
for d in datasets:
    total_assets += len(d.asset_list)

In [41]:
size_mb = total_assets * sys.getsizeof(df)/mb
print(f"Total datasets: {len(datasets)}")
print(f"Total assets uploaded: {total_assets}")
print(f"Total uploaded: {size_mb} MB")
print(f"Total uploaded: {size_mb/1024} GB")

Total datasets: 40
Total assets uploaded: 200
Total uploaded: 47753.23581695557 MB
Total uploaded: 46.63401935249567 GB
